In [ ]:
import os
import csv
from collections import Counter, defaultdict

In [ ]:
# Dossier de travail
dossier_adjudication="Adjudication"
nombre_max_tokens=6500

In [ ]:
# Fonction pour lire un fichier CSV et récupérer les données
def lire_fichier_csv(chemin_fichier, max_tokens):
    donnees = []
    try:
        with open(chemin_fichier, 'r', newline='', encoding='utf-8') as fichier:
            lecteur_csv = csv.DictReader(fichier, delimiter=';', quotechar='"')
            for i, ligne in enumerate(lecteur_csv):
                if i >= max_tokens:
                    break
                donnees.append(ligne)
    except Exception as e:
        print(f"Erreur lors de la lecture de {chemin_fichier}: {e}")
    return donnees

# Fonction pour déterminer l'annotation majoritaire
def determiner_majorite(annotations, label_spacy):
    annotations_filtrees = [a for a in annotations if a and a != "NA"]  # Exclure les NA
    compteur = Counter(annotations_filtrees)
    
    if not annotations_filtrees:
        return "" if annotations.count("NA") >= 3 else label_spacy
    
    majoritaire, nombre = compteur.most_common(1)[0]
    
    # Appliquer la règle de majorité 
    if nombre >= 3:
        return majoritaire
    elif label_spacy in compteur and compteur[label_spacy] >= nombre:
        return label_spacy
    
    return ""

# Fonction pour déterminer Quaero_Maj si Label_Maj == "LOC"
def determiner_quaero_majoritaire(quaeros):
    quaero_filtre = [q for q in quaeros if q and q != "LOC"]
    compteur = Counter(quaero_filtre)
    
    if compteur:
        majoritaires = compteur.most_common()
        max_occurrence = majoritaires[0][1]
        candidats = [q for q, occ in majoritaires if occ == max_occurrence]
        return candidats[0]  # Prendre l'un des majoritaires en cas d'égalité
    return ""

In [ ]:
# Traitement des fichiers et consolidation
for fichier in os.listdir(dossier_adjudication):
    chemin_fichier = os.path.join(dossier_adjudication, fichier)
    print(f"\nTraitement du fichier : {fichier}")
    
    donnees = lire_fichier_csv(chemin_fichier, nombre_max_tokens)
    if not donnees:
        continue
    
    # Extraction des colonnes des annotateurs
    annotateurs_correction = [col for col in donnees[0] if "_Correction" in col]
    annotateurs_quaero = [col for col in donnees[0] if "_Quaero" in col]
    
    donnees_consolidees = []
    
    # Traitement des tokens
    for ligne in donnees:
        token = ligne['Token']
        label_spacy = ligne['Label']
        
        corrections = [ligne[annotateur] for annotateur in annotateurs_correction]
        quaeros = [ligne[annotateur] for annotateur in annotateurs_quaero]
        
        label_maj = determiner_majorite(corrections, label_spacy)
        
        # Traitement de Quaero_Maj si Label_Maj est LOC
        quaero_maj = ""
        if label_maj == "LOC":
            quaero_maj = determiner_quaero_majoritaire(quaeros)
        
        donnees_consolidees.append({
            "Token": token,
            "Label_Maj": label_maj,
            "Quaero_Maj": quaero_maj
        })
    
    # Sauvegarde du fichier consolidé
    chemin_sortie = os.path.join(dossier_adjudication, f"{fichier.replace('.csv', '_consolide.csv')}")
    with open(chemin_sortie, 'w', newline='', encoding='utf-8') as fichier_sortie:
        champs = ["Token", "Label_Maj", "Quaero_Maj"]
        ecrivain_csv = csv.DictWriter(fichier_sortie, fieldnames=champs, delimiter=';', quotechar='"')
        ecrivain_csv.writeheader()
        ecrivain_csv.writerows(donnees_consolidees)
    
    print(f"Fichier consolidé enregistré sous : {chemin_sortie}")